In [1]:
from __future__ import unicode_literals

In [2]:
import sys
sys.path.append('Perfomer_local/keras_layer_normalization')
sys.path.append('Perfomer_local/keras_performer')
sys.path.append('Perfomer_local/keras_position_wise_feed_forward')
sys.path.append('Perfomer_local/tensorflow_fast_attention')

'''
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_layer_normalization')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_performer')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_position_wise_feed_forward')
sys.path.append('D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\tensorflow_fast_attention')
'''

"\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_layer_normalization')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_performer')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\\keras_position_wise_feed_forward')\nsys.path.append('D:\\Code\\Project-AI-JAVA-ANNOTATION-2021\\Performer\\Perfomer_local\tensorflow_fast_attention')\n"

In [3]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
'''
def solve_cudnn_error():
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
            
solve_cudnn_error()
'''

'\ndef solve_cudnn_error():\n    import tensorflow as tf\n    gpus = tf.config.experimental.list_physical_devices(\'GPU\')\n    if gpus:\n        try:\n            # Currently, memory growth needs to be the same across GPUs\n            for gpu in gpus:\n                tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n        except RuntimeError as e:\n            # Memory growth must be set before GPUs have been initialized\n            print(e)\n            \nsolve_cudnn_error()\n'

In [5]:
import csv
def readCSV(file_name):
  errlist=[]
  LBlist=[]
  with open(file_name, newline='') as csvfile:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary

    rows = csv.DictReader(csvfile)
    for row in rows: 
      RL=list(row.values())

      RL = list(map(int, RL))
      errs=RL[1:37]
      LB=RL[37:]
      errlist.append(errs)
      LBlist.append(LB)
  return errlist,LBlist


In [6]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [7]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [8]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [9]:
def parseSentence(x):	
	tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
	tokens=[]
	state="START"
	chrs=""
	for i in range(len(x)):
		#print(ord(x[i]))
		if (ord(x[i])>255):
			inp="U"
		else:
			inp="E"
	
		if state=="START":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
			
		elif state=="ASCII":
			if inp=="E":
				chrs += x[i]
			else:#U
				state="UNICODE"
				tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
				chrs=""
				tokens.append(x[i])
	
		elif state=="UNICODE":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
	if len(chrs)>0:
		tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
	return tokens

In [10]:
def readcode(fname):
    with open(fname) as f:
        data = f.read()
        return data

In [ ]:
class TestTranslate(unittest.TestCase):
 
    @staticmethod
    def _build_token_dict(token_list):
        token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<BOTM>': 5,
            '<BOT>': 6,
            '<EOT>': 7,
            '<BOM>': 8,
            '<EOM>': 9,
            '<EOTM>': 10,
            '<CR>': 11,
        }
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
 
    def test_translate(self):
        print("i am here: " )
        source_tokens = []
        target_errors=[]
        target_LB=[]
        #source_file=[]
        
        Input_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputTxt"
        Location_Output_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputCSV"
        '''
        Input_Path = "Performer/Perfomer_local/Trianing/InputTxt"
        Location_Output_Path = "Performer/Perfomer_local/Trianing/InputCSV"
        '''
 
        InputFiles = sorted(listdir(Input_Path))
        OutputFiles = sorted(listdir(Location_Output_Path))
        #max_files =10
        for f in InputFiles:
          fullpath = join(Input_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            source_tokens.append(parseSentence(readcode(fullpath)))
            #if len(source_tokens)>max_files: break
        for f in OutputFiles:
          fullpath = join(Location_Output_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            err,lb = readCSV(fullpath)
            target_errors.append(err)
            target_LB.append(lb)
            #if len(source_tokens)>max_files: break
        dd=np.asarray(target_errors)
        print("AAAA: ", dd.shape)
        print("aaaa: " , type(target_errors[0][0]))
        target_errors=target_errors[0]  
        target_LB=target_LB[0]     
        print("XXXX: " , len(source_tokens))
        print("YYYY: " , type(target_errors[0][0]))
        print("ZZZZ: " , len(target_LB))

 
        source_tokens2 = []
        target_errors2= []
        target_LB2=[]

        THRESHOLD_FILE_LEN = 64000000
 
        for i in range(len(source_tokens)): 
          #print (i)
          src = source_tokens[i]
          err = target_errors[i]   
          lb = target_LB[i]                   
          if (len(src)<=THRESHOLD_FILE_LEN):
            source_tokens2.append(src)
            target_errors2.append(err)
            target_LB2.append(lb) 
        source_tokens = source_tokens2
        target_errors= target_errors2        
        target_LB= target_LB2        

        print("XXXX2: " , len(source_tokens))
                    
 
        # Generate dictionaries
        source_token_dict = self._build_token_dict(source_tokens)
        
 
        # Add special tokens
        encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
        
        #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
 
        # Padding
        self.sl = max(map(len, encode_tokens))
        
        source_max_len = self.sl
        
 
        encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
        
        
 
        encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
       
        
        #print("encode_input:", encode_input)
        
        #print("len(source_token_dict):", len(source_token_dict))
        
        
        #decode_output_one_hot_encoded = to_categorical(decode_output)
        #print("one-hot-decode_output:", decode_output_one_hot_encoded)
        #decode_output =  list(decode_output_one_hot_encoded)
        token_num =len(source_token_dict)
        print(type(token_num))
        #Build & fit model
        #Set model para    
        model = tfr.get_model(
            max_input_len= (source_max_len),
            errNum=36,
            lbNum=192,
            token_num=len(source_token_dict),
            embed_dim=32, #32,
            encoder_num=2, #2 max = 6
            head_num=2,#4
            hidden_dim=64, #128
            dropout_rate=0.05 #0.05
        )
        losses = {    
            "error_feed_forward_output1": "categorical_crossentropy",
            "error_feed_forward_output2": "mean_squared_error",            
        }
        lossWeights = {"error_feed_forward_output1": 1.0,"error_feed_forward_output2": 1.0}
        model.compile('adam', loss=losses, loss_weights=lossWeights)
        
        print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
        
 
        #x=[np.array(encode_input * 1)]
        #y=[np.array(target_errors * 1),np.array(target_LB * 1)]
       
        #print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)
        
        ####  Split the data set into train and test_model
        x=np.array(encode_input)
        y=list(zip(np.array(target_errors), np.array(target_LB)))


        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

        y_train = list(zip(*y_train))
        y_train[0] = np.array(y_train[0])
        y_train[1] = np.array(y_train[1])

        
        y_test = list(zip(*y_test))
        y_test[0] = np.array(y_test[0])
        y_test[1] = np.array(y_test[1])
               
        
        saveTestTrainData("train_models/x_train_500.npy", x_train)
        saveTestTrainData("test_models/x_test_500.npy", x_test)
        saveTestTrainData("train_models/y_train0_500.npy", y_train[0])#target
        saveTestTrainData("train_models/y_train1_500.npy", y_train[1])#lb
        saveTestTrainData("test_models/y_test0_500.npy", y_test[0])
        saveTestTrainData("test_models/y_test1_500.npy", y_test[1])
    
        ####
        #if you only need to train model don't use this
        
        model.fit(
            x=x_train,
            y=y_train,
            epochs=500, #100 200 500
            batch_size=32,
        )
        
        model.save("test_model1.h5")
        saveDictionary(source_token_dict, 'source_token_dict.pickle')
        
        model, source_token_dict = load("test_model1.h5")

        # Predict
        x_train_loaded = loadTestTrainData("train_models/x_train_500.npy") 
        y_train_loaded = [  loadTestTrainData("train_models/y_train0_500.npy"),
                          loadTestTrainData("train_models/y_train1_500.npy") ]

        x_test_loaded =  loadTestTrainData("test_models/x_test_500.npy") 
        y_test_loaded = [ loadTestTrainData("test_models/y_test0_500.npy"),
                         loadTestTrainData("test_models/y_test1_500.npy")]
        
        out1, out2 = tfr.decode(
            model,
            #encode_input,
            x_test_loaded,max_len=source_max_len
        )
        print("out1: ", out1)
        #print("target_errors: ",target_errors) 
        print("target_errors: ", y_test_loaded[0])        
        print("out2: ", out2)
        #print("target_LB: ", target_LB)
        print("target_LB: ", y_test_loaded[1])
        
    def getsource_max_lan(self):
        return self.sl

In [12]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [13]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [14]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [15]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [16]:
def load(model_name):
        import sys
        sys.path.append('Perfomer_local/keras_position_wise_feed_forward')
        sys.path.append('Perfomer_local/tensorflow_fast_attention')
        sys.path.append('Perfomer_local/keras_performer')
        #sys.path.append('/Perfomer_local/keras_pos_embed')

        from keras_performer import performer
        from tensorflow import keras
        from keras_embed_sim import EmbeddingRet, EmbeddingSim
        from keras_pos_embd import TrigPosEmbedding
        from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
        from keras_position_wise_feed_forward.feed_forward import FeedForward  

        co = performer.get_custom_objects()

        model = keras.models.load_model(model_name, custom_objects= co)
        source_token_dict = loadDictionary('source_token_dict.pickle')
       # t = loadDictionary(target_token_dict, 'target_token_dict.pickle')
       # t_inv = loadDictionary(target_token_dict_inv, 'target_token_dict_inv.pickle')
        return model, source_token_dict,

In [17]:
x=TestTranslate()
x.test_translate()

i am here: 
AAAA:  (1, 359, 36)
aaaa:  <class 'list'>
XXXX:  359
YYYY:  <class 'int'>
ZZZZ:  359
XXXX2:  359
<class 'int'>
In get_model: encoder_input:  (None, 2919)
In get_model: encoder_embed:  (None, 2919, 32)
Start Warpping................................
OOOO: KerasTensor(type_spec=TensorSpec(shape=(None, 2919, 32), dtype=tf.float32, name=None), name='Encoder-Embedding/add_1:0', description="created by layer 'Encoder-Embedding'")
embed_dim: 32
head_num: 2
dropout_rate: 0.05
masked: False
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
dim:  16
query:  Tensor("self_attention_1/query/einsum/Einsum:0", shape=(None, 2919, 2, 16), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
Start Warpping.......................

Epoch 1/500
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
dim:  16
query:  Tensor("model/self_attention_1/query/einsum/Einsum:0", shape=(None, 2919, 2, 16), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
dim:  16
query:  Tensor("model/self_attention_3/query/einsum/Einsum:0", shape=(None, 2919, 2, 16), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_

11/11 [==============================] - 13s 1s/step - loss: 56.0194 - error_feed_forward_output1_loss: 5.9670 - error_feed_forward_output2_loss: 50.0524
Epoch 39/500
11/11 [==============================] - 13s 1s/step - loss: 45.6231 - error_feed_forward_output1_loss: 5.8740 - error_feed_forward_output2_loss: 39.7491
Epoch 40/500
11/11 [==============================] - 13s 1s/step - loss: 47.6821 - error_feed_forward_output1_loss: 5.7908 - error_feed_forward_output2_loss: 41.8913
Epoch 41/500
11/11 [==============================] - 13s 1s/step - loss: 42.3131 - error_feed_forward_output1_loss: 5.8117 - error_feed_forward_output2_loss: 36.5014
Epoch 42/500
11/11 [==============================] - 13s 1s/step - loss: 48.7615 - error_feed_forward_output1_loss: 5.8240 - error_feed_forward_output2_loss: 42.9375
Epoch 43/500
11/11 [==============================] - 13s 1s/step - loss: 45.4454 - error_feed_forward_output1_loss: 6.1365 - error_feed_forward_output2_loss: 39.3089
Epoch 44/50

11/11 [==============================] - 12s 1s/step - loss: 47.0189 - error_feed_forward_output1_loss: 5.6986 - error_feed_forward_output2_loss: 41.3203
Epoch 88/500
11/11 [==============================] - 12s 1s/step - loss: 51.2849 - error_feed_forward_output1_loss: 5.6631 - error_feed_forward_output2_loss: 45.6218
Epoch 89/500
11/11 [==============================] - 13s 1s/step - loss: 58.1093 - error_feed_forward_output1_loss: 5.8543 - error_feed_forward_output2_loss: 52.2550
Epoch 90/500
11/11 [==============================] - 12s 1s/step - loss: 55.4105 - error_feed_forward_output1_loss: 5.7969 - error_feed_forward_output2_loss: 49.6136
Epoch 91/500
11/11 [==============================] - 12s 1s/step - loss: 44.6461 - error_feed_forward_output1_loss: 5.5945 - error_feed_forward_output2_loss: 39.0516
Epoch 92/500
11/11 [==============================] - 12s 1s/step - loss: 52.6198 - error_feed_forward_output1_loss: 5.5964 - error_feed_forward_output2_loss: 47.0235
Epoch 93/50

11/11 [==============================] - 12s 1s/step - loss: 51.3152 - error_feed_forward_output1_loss: 5.6967 - error_feed_forward_output2_loss: 45.6185
Epoch 136/500
11/11 [==============================] - 12s 1s/step - loss: 49.9185 - error_feed_forward_output1_loss: 5.6492 - error_feed_forward_output2_loss: 44.2693
Epoch 137/500
11/11 [==============================] - 12s 1s/step - loss: 49.5307 - error_feed_forward_output1_loss: 6.0351 - error_feed_forward_output2_loss: 43.4957
Epoch 138/500
11/11 [==============================] - 12s 1s/step - loss: 48.6353 - error_feed_forward_output1_loss: 5.6337 - error_feed_forward_output2_loss: 43.0017
Epoch 139/500
11/11 [==============================] - 12s 1s/step - loss: 48.6511 - error_feed_forward_output1_loss: 5.5942 - error_feed_forward_output2_loss: 43.0569
Epoch 140/500
11/11 [==============================] - 12s 1s/step - loss: 49.1818 - error_feed_forward_output1_loss: 5.6844 - error_feed_forward_output2_loss: 43.4974
Epoch 

...

11/11 [==============================] - 12s 1s/step - loss: 51.7495 - error_feed_forward_output1_loss: 5.4852 - error_feed_forward_output2_loss: 46.2643
Epoch 376/500
11/11 [==============================] - 12s 1s/step - loss: 53.6657 - error_feed_forward_output1_loss: 5.4698 - error_feed_forward_output2_loss: 48.1959
Epoch 377/500
11/11 [==============================] - 12s 1s/step - loss: 47.1720 - error_feed_forward_output1_loss: 5.2904 - error_feed_forward_output2_loss: 41.8817
Epoch 378/500
11/11 [==============================] - 12s 1s/step - loss: 54.3381 - error_feed_forward_output1_loss: 5.3631 - error_feed_forward_output2_loss: 48.9750
Epoch 379/500
11/11 [==============================] - 12s 1s/step - loss: 50.3046 - error_feed_forward_output1_loss: 5.3435 - error_feed_forward_output2_loss: 44.9611
Epoch 380/500
11/11 [==============================] - 13s 1s/step - loss: 53.6918 - error_feed_forward_output1_loss: 5.4380 - error_feed_forward_output2_loss: 48.2538
Epoch 

11/11 [==============================] - 13s 1s/step - loss: 51.1006 - error_feed_forward_output1_loss: 5.5705 - error_feed_forward_output2_loss: 45.5301
Epoch 424/500
11/11 [==============================] - 13s 1s/step - loss: 44.1873 - error_feed_forward_output1_loss: 5.0214 - error_feed_forward_output2_loss: 39.1659
Epoch 425/500
11/11 [==============================] - 12s 1s/step - loss: 45.8692 - error_feed_forward_output1_loss: 5.4838 - error_feed_forward_output2_loss: 40.3854
Epoch 426/500
11/11 [==============================] - 12s 1s/step - loss: 50.9649 - error_feed_forward_output1_loss: 5.2307 - error_feed_forward_output2_loss: 45.7342
Epoch 427/500
11/11 [==============================] - 13s 1s/step - loss: 48.7623 - error_feed_forward_output1_loss: 5.2710 - error_feed_forward_output2_loss: 43.4913
Epoch 428/500
11/11 [==============================] - 13s 1s/step - loss: 42.3391 - error_feed_forward_output1_loss: 5.1780 - error_feed_forward_output2_loss: 37.1611
Epoch 

11/11 [==============================] - 12s 1s/step - loss: 51.7533 - error_feed_forward_output1_loss: 5.6507 - error_feed_forward_output2_loss: 46.1026
Epoch 472/500
11/11 [==============================] - 14s 1s/step - loss: 49.1490 - error_feed_forward_output1_loss: 5.5372 - error_feed_forward_output2_loss: 43.6118
Epoch 473/500
11/11 [==============================] - 14s 1s/step - loss: 54.0060 - error_feed_forward_output1_loss: 5.4261 - error_feed_forward_output2_loss: 48.5799
Epoch 474/500
11/11 [==============================] - 14s 1s/step - loss: 50.1239 - error_feed_forward_output1_loss: 5.2170 - error_feed_forward_output2_loss: 44.9069
Epoch 475/500
11/11 [==============================] - 13s 1s/step - loss: 43.6904 - error_feed_forward_output1_loss: 5.2838 - error_feed_forward_output2_loss: 38.4066
Epoch 476/500
11/11 [==============================] - 13s 1s/step - loss: 52.7742 - error_feed_forward_output1_loss: 5.7045 - error_feed_forward_output2_loss: 47.0697
Epoch 

C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOO

In [18]:
# the following funcs is make socre to model 
testmodel = "test_model1.h5"

In [19]:
def loadTestdata():
    import numpy as np
    #load model and dic ps. dic is not use
    model, source_token_dict = load(testmodel)
    #load
    x_test_loaded = loadTestTrainData("test_models/x_test_500.npy") 
    y_test_loaded = [ loadTestTrainData("test_models/y_test0_500.npy"), loadTestTrainData("test_models/y_test1_500.npy")] 
    out1, out2 = tfr.decode(model,x_test_loaded,max_len=x.getsource_max_lan()) 
    #print((y_test_loaded[0][1])) #Error_type #vs out1
    #print((y_test_loaded[1][1])) #Line_Block #vs out2
    #print((out1[1]))#prob upper then 0.5
    #print((out2[1]))#prob upper then 0.5
    #print(int(out2))#prob lb
    test_ep = np.around(out1)
    test_lb = np.around(out2)
    ans_ep = np.around(y_test_loaded[0])
    ans_lb = np.around(y_test_loaded[1])
    print(test_ep[1])
    print(test_lb[1])
    print(ans_ep[1])
    print(ans_lb[1])
    #scoreTest(test_ep,test_lb,ans_ep,ans_lb) #go to test the score

In [20]:
def scoreTest(test_ep,test_lb,ans_ep,ans_lb):
    
    for i in range()

SyntaxError: invalid syntax (<ipython-input-20-734122e03e9d>, line 3)

In [21]:
loadTestdata()

query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
query_input: XXXXXXX  (None, 2919, 32)
source_input: XXXXXXX  (None, 2919, 32)
Query: XXXXXXX  (None, 2919, 2, 16)
Key: XXXXXXX  (None, 2919, 2, 16)
Value: XXXXXXX  (None, 2919, 2, 16)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2919, None, 2, 16)
KS OOOOOOOOOOOO